### Text Classification

In [ ]:
%matplotlib inline

import pandas as pd

In [ ]:
#https://www.kaggle.com/zynicide/wine-reviews/data

wine_df = pd.read_csv('data/wine_reviews.csv')


<div class="alert alert-info">
<h3> Your turn</h3>
<p> What's in the dataset?
</div>


In [ ]:
wine_df.head()

In [ ]:
wine_df['description'][:5]

![google_search.png](images/google_search.png)

In [ ]:
pd.set_option('display.max_colwidth', 120)



In [ ]:
wine_df['description'][:5]

In [ ]:
wine_df.head()

### Turning words in to features

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
vectorizer = CountVectorizer(lowercase=True,
                             ngram_range = (1,1),
                             stop_words  = 'english',
                             max_df      = .60,
                             min_df      = .01,
                             max_features = None)

In [ ]:
vectorizer.fit(wine_df['description'])

In [ ]:
len(vectorizer.get_feature_names())

In [ ]:
review_word_counts = vectorizer.transform(wine_df['description'])

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
nb_classifier = MultinomialNB()


In [ ]:
nb_classifier.fit(review_word_counts, wine_df['rating'])

In [ ]:
nb_classifier.coef_

In [ ]:
coeficients = pd.Series(nb_classifier.coef_[0], 
                        index=vectorizer.get_feature_names())

In [ ]:
coeficients.sort_values()[:20]

In [ ]:
coeficients.sort_values(ascending=False)[:20]

<div class="alert alert-info">
<h3> Your turn</h3>
<p> New groups! Load up the group spreadsheet and find your group. Working in your group, use the "ge_speeches.json" file to determine what are the most distinguishing words used by Hillary Clinton and Donald Trump during the 2016 election. Do this in a new notebook!

</div>


In [ ]:
nb_classifier.predict(review_word_counts)

In [ ]:
wine_df['prediction']  = nb_classifier.predict(review_word_counts)

In [ ]:
pd.crosstab(wine_df['rating'], wine_df['prediction'])

In [ ]:
nb_classifier.predict_proba(review_word_counts)

In [ ]:
predict_df = pd.DataFrame(nb_classifier.predict_proba(review_word_counts), 
                          columns=nb_classifier.classes_)

In [ ]:
predict_df.head()

In [ ]:
wine_df_prediction = pd.concat([wine_df, predict_df], axis = 1)

In [ ]:
wine_df_prediction.sort_values('High', ascending=False)[['description','points']].head(15)

In [ ]:
wine_df_prediction.sort_values('Low', ascending=False)[['description','points']].head(15)

<div class="alert alert-info">
<h3> Your turn</h3>
<p> In your groups, how well do your models fit? What is the most Trumpish Trump speech? What is the least?

</div>



### What about overfitting?

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(wine_df, test_size=0.2)

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
vectorizer.fit(train['description'])

In [ ]:
X_train = vectorizer.transform(train['description'])

In [ ]:
nb_classifier.fit(X_train, train['rating'])

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score



In [ ]:
print(accuracy_score(train['rating'],
                     nb_classifier.predict(X_train)))



In [ ]:
print(confusion_matrix(train['rating'], 
                      nb_classifier.predict(X_train)))


In [ ]:
print(classification_report(train['rating'],
                       nb_classifier.predict(X_train)))



Precision: % of selected items that are correct 

Recall: % of correct items that are selected


In [ ]:
test_wf         = vectorizer.transform(test['description'])
test_prediction = nb_classifier.predict(test_wf)

In [ ]:
print(accuracy_score(test['rating'], test_prediction))

In [ ]:
print(classification_report(test['rating'], test_prediction))


In [ ]:
vectorizer = CountVectorizer(lowercase=True,
                             ngram_range = (1,1),
                             stop_words = 'english',
                             max_df = .60,
                             min_df = 5,
                             max_features = None)

In [ ]:
vectorizer.fit(train['description'])
X_train = vectorizer.transform(train['description'])
nb_classifier.fit(X_train, train['rating'])


In [ ]:
print(accuracy_score(train['rating'],
                     nb_classifier.predict(X_train)))



In [ ]:

print(accuracy_score(test['rating'],
                     nb_classifier.predict(vectorizer.transform(test['description']))))


<div class="alert alert-info">
<h3> Your turn</h3>
<p> What happens to your model if you change some of the parameters for your vectorizer? Be sure to spit the data between train and test!

</div>




### What about a different model?

![](images/knn1.png)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors = 3)

In [ ]:
knn_classifier.fit(review_word_counts, wine_df['rating'])

In [ ]:
knn_prediction = knn_classifier.predict(review_word_counts)

In [ ]:

print(accuracy_score(test['rating'], knn_prediction))



In [ ]:
print(classification_report(test['rating'], knn_prediction))

<div class="alert alert-info">
<h3> Your turn</h3>
<p> What does a k-nearest neigbhor for your speech dataset look like? How does the accuracy compare?
</div>





![](images/knn2.png)

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors = 15)

### But what's the best fitting model?

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'n_neighbors' : (2, 3, 4)}
              

In [ ]:
grid = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)

In [ ]:
grid.fit(review_word_counts,
         wine_df['rating'])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
pipeline = Pipeline([
                     ('vectorizer' , CountVectorizer()),
                     ('classifier' , MultinomialNB())
                    ])

In [ ]:
parameters = {'vectorizer__max_df' : (.2, .4),
              'vectorizer__min_df' : (100, 150)
             }
              

In [ ]:
grid_search = GridSearchCV(pipeline,
                           parameters,
                           n_jobs = -1,
                           cv = 3,
                           verbose = 1)

In [ ]:
grid_search.fit(wine_df['description'],
                wine_df['rating'])

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_estimator_

In [ ]:
parameters = {'vectorizer__max_df'      : [.1, .15, .2, .25],
              'vectorizer__min_df'      : [25, 50, 100],
              'vectorizer__stop_words'  : [None, 'english'],
              'vectorizer__ngram_range' : [(1,1), (1,2)]
             }
              

In [ ]:
grid_search.best_score_

In [ ]:
grid_search = GridSearchCV(pipeline,
                           parameters,
                           n_jobs = -1,
                           cv = 5,
                           verbose = 1)

In [ ]:
grid_search.fit(wine_df_extremes['description'],
                wine_df_extremes['rating'])

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.best_estimator_.get_params